## Data Compression Problem

Data Compression คือ การบีบอัดข้อมูลให้เล็กลง ยกตัวอย่างเช่น ในกรณีที่เราต้องการเก็บข้อมูลประเภท String ที่ประกอบด้วยตัวอักษรต่างๆ <br>
เราสามารถกำหนดขนาดของข้อมูลได้ทั้งหมด 2 รูปแบบ ได้แก่
1. <span style="font-weight: bold;">ใช้ ASCII Table</span>
หมายความว่า ทุกๆ ตัวอักษรจะใช้ขนาด 8 bits เสมอในการเก็บ ทำให้ปัญหาคือ <span style="color: yellow;">ข้อมูลที่เก็บมีขนาดที่ใหญ่ แม้ตัวอักษรจะน้อยก็ตาม</span>

<img src="https://www.sciencebuddies.org/cdn/references/ascii-table.png" style="width:600px;">


2. <span style="font-weight: bold;">การกำหนดขนาดขึ้นมาเอง (Own fixed-length size codes)</span>>
หมายความว่า เราดูตัวอักษรที่อยู่ใน String ที่เราจะเก็บก่อน แล้วดู Unique items  <br> จากนั้นเรากำหนดรหัสแทนแต่ละตัวอักษรขึ้นมาเอง เช่น ถ้าหากมี A-G
เราก็จะกำหนดเป็น 3 bits (แทน A = 000 จนถึง G = 110)

<span style="color: yellow;">แต่ปัญหาคือ</span> ถ้าหากเรามีตัวอักษรที่เพิ่มมาอีกไม่กี่ตัว แล้วมันทำให้ทุกตัวเพิ่มรหัสเป็น 4-bit แทน
มันจะเปลืองเนื้อที่มาก 

3. <span style="font-weight: bold;">Variable-length encoding</span>

คือการแปลงให้เป็นรหัสที่มีขนาดไม่เท่ากัน โดยอาศัยการแปลงจากอักษรเป็นโค้ด และเข้าถึงการแปลโดย Binary Tree <br> 
ที่มีหลักการคือโหนดลูกซ้ายเป็น 0 โหนดลูกขวาเป็น 1 
อีกหลักการคือต้องเป็น Prefix Free code ซึ่งหมายถึง ห้ามมีเลขที่ตัวหน้าซ้ำกัน ยกตัวอย่างเช่น หากมี 01 เป็น A แล้ว <br>
จะไม่สามารถมี 011 ได้ เพราะมันจะถูกแปลเป็น A ดังนั้น Prefix ตัวก่อนหน้านั้นห้ามซ้ำกันเลย

ต่อมา Samuel Morse ได้เสนอให้แนวคิดดังกล่าว มีการหาความถี่ของตัวที่ใช้บ่อยก่อน (หา Frquency ของ Unique items) <br>
ถ้าเราใช้บ่อยจะทำให้ขนาดข้อมูลของรหัสเราสั้นในการแทนรหัส <br>

Fun fact : รหัส Morse ที่คิดค้นขึ้นก็ใช้หลักการความถี่ตัวอักษรที่ใช้บ่อย เช่น สระ AEIOU จะเป็นรหัสที่สั้นกว่า

จากแนวคิดทั้งหมดของ Variable-length encoding เราจะเรียกว่า <span style="color: orange; font-weight:bold;">Huffman's algorithm</span>

## Data Compression Problem with Own fixed-length size codes

In [54]:
import math

def UniqueChars(string):
    # Create a dictionary to store unique characters
    codewords = dict()
    for char in string:
        if char not in codewords:
            codewords[char] = 0

    # Check len of unique characters to calculate bits for every codewords
    # len(string) = 2^n
    # n = log2(len(string))
    bits = math.ceil(math.log2(len(codewords)))

    code = 0
    for char in codewords:
        codewords[char] = bin(code)[2:].zfill(bits)
        code += 1

    return codewords

def encodedString(string, codewords):
    encoded = ""
    for char in string:
        encoded += codewords[char]
    return encoded

def decodedString(encoded, codewords):
    decoded = ""
    i = 0
    while i < len(encoded):
        for char in codewords:
            if encoded[i:i+len(codewords[char])] == codewords[char]:
                decoded += char
                i += len(codewords[char])
                break
    return decoded

def CompressionRation(string, encoded):
    original = len(string) * 8
    compressed = len(encoded)
    ratio = ((original - compressed) / original) * 100
    return ratio
   
def fixed_length_compression(string):
    codewords = UniqueChars(string)
    print("\n-- Codewords for each character --")
    for value in codewords:
        print(value, ":", codewords[value])

    encoded = encodedString(string, codewords)
    print("\n-- Encoded string --")
    print(encoded)

    ratio = CompressionRation(string, encoded)
    print("\n-- Compression ratio --")
    print(ratio ,"%")

    decoded = decodedString(encoded, codewords)
    print("\n-- Decoded string --")
    print(decoded)

In [55]:
## Test Case 1 ##
string = "abracadabra"
fixed_length_compression(string)


-- Codewords for each character --
a : 000
b : 001
r : 010
c : 011
d : 100

-- Encoded string --
000001010000011000100000001010000

-- Compression ratio --
62.5 %

-- Decoded string --
abracadabra


In [57]:
## Test Case 2 ##
string = "bling bang bang born"
fixed_length_compression(string)


-- Codewords for each character --
b : 0000
l : 0001
i : 0010
n : 0011
g : 0100
  : 0101
a : 0110
o : 0111
r : 1000

-- Encoded string --
00000001001000110100010100000110001101000101000001100011010001010000011110000011

-- Compression ratio --
50.0 %

-- Decoded string --
bling bang bang born


## Practice II : Huffman's Algorithm

Reference 1 : https://www.javatpoint.com/huffman-coding-using-python \
Reference 2 : https://www.geeksforgeeks.org/huffman-coding-greedy-algo-3/

### หลักการการทำ Huffman's Algorithm

ในอัลกอริทึมนี้จะมีต้นไม้เพื่อเข้าถึงรหัส ซึ่งเรียกว่า <span style="color: hotpink;">"Huffman tree"</span> ส่วนตัวโค้ดจะเรียกว่า <span style="color:lime">"Huffman code"</span>

Step 1 : หาความถี่ของแต่ละตัวอักษร จากนั้นสร้าง Tree ที่มีโหนดเดี่ยวจำนวน n ต้น โดยให้กำหนดค่าความถี่ของโหนดและ
ทำสัญลักษณ์ว่าตัวอักษรอะไร

Step 2 : ทำการวนซ้ำ จับ 2 trees ที่มี weight น้อยที่สุดในแต่ละรอบมาประกอบเป็น Tree ใหม่ โดยสร้างเป็น <br>
Tree ที่มีโหนดลูกซ้ายขวาเป็นตัวที่เราจับมา และนำค่า Weight มารวมกันเป็นโหนดพ่อแม่ จากนั้นมันจะนับเป็น Tree ตัวใหม่
จากนั้นวนซ้ำทำไปเรื่อยๆ จนกว่าจะไม่เหลือต้นไม้ให้จับคู่กัน

<img src="https://cgi.luddy.indiana.edu/~yye/c343-2019/images/Huffman-tree-Fig5.24.png">

In [45]:
# Huffman's Tree

# สร้าง class สำหรับเก็บข้อมูลของ node แต่ละตัว
class NodeTree():
    def __init__(self, freq, char, left=None, right=None):
        self.left = left
        self.right = right
        self.freq = freq
        self.char = char
        self.code = ""

    def setCode(self, dir):
        self.code = dir

    def children(self):
        return (self.left, self.right)
    
    def __str__(self):
        return '%s_%s_%s' % (self.char, self.freq, self.code)
    
    def __lt__(self, next):
        return self.freq < next.freq
    
# เอาไว้หาความถี่จากตัวอักษรใน string
def CalculateFrequency(string):
    freq_list = {}
    for char in string:
        if char not in freq_list:
            freq_list[char] = 1
        else:
            freq_list[char] += 1
    return freq_list

# หา code ของแต่ละตัวอักษร
def EncodeCharacter(node, codewords, value=""):

    newValue = value + str(node.code)

    if (node.left):
        EncodeCharacter(node.left, codewords, newValue)
    if (node.right):
        EncodeCharacter(node.right, codewords, newValue)

    if (not node.left and not node.right):
        codewords[node.char] = newValue
    
    return codewords

# ทำการ encode ข้อความทั้งหมด ให้กลายเป็นข้อความที่เข้ารหัสแล้ว
def EncodeString(string, codewords):
    encodedString = ""
    for char in string:
        encodedString += codewords[char]
    return encodedString

# หาขนาดที่ลดลงหลังจากที่ทำการ encode ด้วย Huffman's Tree แล้ว
def CompressionRatio(string, codewords):
    totalBits = len(string) * 8
    compressedBits = 0
    
    for char in string:
        compressedBits += len(codewords[char])

    return (totalBits - compressedBits) / totalBits * 100

# ทำการ decode ข้อความที่เข้ารหัสแล้ว ให้กลายเป็นข้อความปกติ
def HuffmanDecoding(encodedString, codewords):
    decodedString = ""
    code = ""
    for char in encodedString:
        # ทำการ Brute Force เพื่อหาว่ามันกลายเป็น code ที่มีในตารางแล้วหรือไม่
        code += char
        for key in codewords:
            if codewords[key] == code:
                decodedString += key
                code = ""
                break
    return decodedString

def HuffmanEncoding(string):
    # 1. หาความถี่ของตัวอักษรใน string
    freq_list = CalculateFrequency(string)  
    characters, freqs = freq_list.keys() , freq_list.values()
    print("Characters : ", characters)
    print("Frequencies : ", freqs)
      
    nodes = []
      
    # สร้าง node จากความถี่ของตัวอักษร แล้วเก็บไว้เพื่อจับคู่
    for char in characters:  
        currNode = NodeTree(freq_list[char], char)
        nodes.append(currNode)
      
    while len(nodes) > 1:  
        
        # ทำการเรียงความถี่จากน้อยไปมาก
        sorted_nodes = sorted(nodes, key = lambda x: x.freq)  

        # จับคู่ 2 ตัวที่น้อยที่สุด
        right = sorted_nodes[0]
        left = sorted_nodes[1]
      
        # กำหนด code ให้ node ที่มากกว่า
        left.setCode(0)
        right.setCode(1)
      
        # สร้าง node ใหม่ โดยให้ความถี่เท่ากับผลรวมของ node ทั้ง 2 ตัวที่น้อยที่สุด 
        newNode = NodeTree(left.freq + right.freq , left.char + right.char , left, right) 
      
        # ลบ node ที่เก่าออก แล้วใส่โหนดใหม่เข้าไปแทน เพื่อจับคู่ต่อไป
        nodes.remove(left)
        nodes.remove(right)
        nodes.append(newNode)

    # หา code ของแต่ละตัวอักษร
    codewords = dict()
    codewords = EncodeCharacter(nodes[0],codewords)

    print('\n-- Code for each characters --')
    for char in codewords:
        print(char, ":", codewords[char])

    # ทำการ encode ข้อความทั้งหมด ให้กลายเป็นข้อความที่เข้ารหัสแล้ว
    encodedString = EncodeString(string, codewords)
    print('\n-- Encoded String --')
    print(encodedString)

    # หาขนาดที่ลดลงหลังจากที่ทำการ encode ด้วย Huffman's Tree แล้ว
    print('\n-- Compression Ratio --')
    print(CompressionRatio(string, codewords), "%")
    
    # ทดสอบการ Decode ว่าถูกต้องหรือไม่
    print('\n-- Decoded String --')
    print(HuffmanDecoding(encodedString, codewords))

In [47]:
## Test-case 1 ##

text_string = "a greedy algorithm is a simple and intuitive algorithm that is used in optimization problems"
HuffmanEncoding(text_string)

Characters :  dict_keys(['a', ' ', 'g', 'r', 'e', 'd', 'y', 'l', 'o', 'i', 't', 'h', 'm', 's', 'p', 'n', 'u', 'v', 'z', 'b'])
Frequencies :  dict_values([7, 14, 3, 4, 6, 3, 1, 4, 5, 12, 8, 3, 5, 5, 3, 4, 2, 1, 1, 1])

-- Code for each characters --
t : 0000
r : 00010
p : 00011
  : 001
a : 0100
h : 01010
d : 01011
i : 011
e : 1000
g : 10010
b : 100110
z : 100111
s : 1010
m : 1011
o : 1100
v : 110100
y : 110101
u : 11011
n : 1110
l : 1111

-- Encoded String --
01000011001000010100010000101111010100101001111100101100000100110000010101011001011101000101000011010011101100011111110000010100111001011001011111000001101101100000111101001000001010011111001011000001001100000101010110010000010100100000000101110100011101110101000010110010111110001110000011000001110110111001110100000001111001110001000110001011001001101111100010111010

-- Compression Ratio --
50.0 %

-- Decoded String --
a greedy algorithm is a simple and intuitive algorithm that is used in optimization problems


In [49]:
## Test Case 2 ##
text_string = "abracadabra"
HuffmanEncoding(text_string)

Characters :  dict_keys(['a', 'b', 'r', 'c', 'd'])
Frequencies :  dict_values([5, 2, 2, 1, 1])

-- Code for each characters --
r : 000
b : 001
d : 010
c : 011
a : 1

-- Encoded String --
10010001011101010010001

-- Compression Ratio --
73.86363636363636 %

-- Decoded String --
abracadabra


---